In [68]:
import os
import logging
import json
import psycopg2

def connect_to_database():
    # Define your database connection parameters
    db_params = {
        "host": "localhost",
        "database": "blockchain",
        "user": "postgres",
        "password": "auth2020"
    }

    try:
        # Connect to the database
        conn = psycopg2.connect(**db_params)
        print("Connected to the database successfully!")
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")
        return None

def get_most_recent_transaction_exec_id(conn):
    try:
        with conn.cursor() as cursor:
            # Query to retrieve the most recent transaction_exec_id
            get_transaction_exec_id_query = """
                SELECT transaction_exec_id
                FROM transaction_header_exec
                ORDER BY transaction_exec_id DESC
                LIMIT 1;
            """
            cursor.execute(get_transaction_exec_id_query)
            result = cursor.fetchone()
            if result:
                return result[0]
            else:
                print("No transaction_exec_id found.")
                return None
    except psycopg2.Error as e:
        print(f"Error retrieving transaction_exec_id: {e}")
        return None

def main():
    block = '6024636'  # Change this number if you want to load a different file
    file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

    try:
        # Check if the file exists
        if os.path.exists(file_path):
            print(f"Found JSON file: {file_path}")

            # Load JSON data from the file
            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)

                # Connect to the database
                conn = connect_to_database()

                # Get the most recent transaction_exec_id
                transaction_exec_id = get_most_recent_transaction_exec_id(conn)

                if transaction_exec_id:
                    # Initialize an empty list to store insert queries
                    insert_queries = []

                    # Iterate through the messages
                    for msg in json_data['tx']['body']['messages'][0]['msgs']:
                        # Extract data from the given JSON structure
                        type_value = msg["@type"]
                        delegator_address_value = msg["delegator_address"]
                        validator_address_value = msg["validator_address"]
                        amount_value = msg["amount"]["amount"]
                        denom_value = msg["amount"]["denom"]

                        # Construct the insert query for the current message
                        msg_insert_query = f"""
                            INSERT INTO transaction_detail_msg_exec (transaction_exec_id, type, delegator_address, validator_address, denom, amount)
                            VALUES ('{transaction_exec_id}', '{type_value}', '{delegator_address_value}', '{validator_address_value}', '{denom_value}', {amount_value});
                        """
                        insert_queries.append(msg_insert_query)

                    # Execute the insert queries
                    with conn.cursor() as cursor:
                        for query in insert_queries:
                            cursor.execute(query)
                        conn.commit()  # Commit the transaction

                    print("Data inserted successfully!")
                else:
                    print("No transaction_exec_id found. Data not inserted.")

            # Close the database connection
            conn.close()
        else:
            print(f"JSON file not found: {file_path}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()



Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/6024636.json
Connected to the database successfully!
Data inserted successfully!
